In [ ]:
%pip install -q langchain langchain-community langchain-openai langchain-chroma python-dotenv pypdf bs4

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Establecemos los modelos que vamos a usar
EMBD_MODEL="text-embedding-3-small"
LLM_MODEL="gpt-4o-mini"

#### Indexing

##### 1. Load Documents

In [3]:
# from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_community.document_loaders import WebBaseLoader

document_path = "resources/00 SCRUMstudy-SBOK-Guide-3rd-edition-Spanish.pdf"

# loader = PyPDFLoader(file_path=document_path)
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
document = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


##### 2. Split Documents

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Split document loaded into a list of documents List[Document]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048)
docs = text_splitter.split_documents(document)
len(docs)

33

##### 3. Embeddings and Store

In [5]:
# Embeddings
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma

emdb = OpenAIEmbeddings(model=EMBD_MODEL)
vectorstore = Chroma.from_documents(documents=docs, embedding=emdb)

#### Retrieve and Generate

##### Retrieve

In [34]:
retriever = vectorstore.as_retriever()
question = 'What are the approaches to Task Decomposition?'
retrieved_docs = retriever.invoke(question)
retrieved_docs

[Document(metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performa

##### Generate answer

In [37]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

qa_system_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {input} 
Context: {context} 
Answer:
"""


qa_prompt = ChatPromptTemplate(
    [
        ("system", qa_system_prompt),
        ("human", "{input}"),
    ],
    input_variables=["context", "input"]
)

llm = ChatOpenAI(model=LLM_MODEL)

qa_chain = {
    "context": retriever,
    "input": RunnablePassthrough()
} | qa_prompt | llm | StrOutputParser()

# question = 'En que consiste el enfoque Task Decomposition?'

response = qa_chain.invoke(question)

In [38]:
response

'No sé quién es el mejor jugador de fútbol.'